# HyLoRADA: Software Evolution & Reengineering

## Novel Contributions
1. **Orthogonal initialization** - Prevents rank collapse
2. **Gated magnitude** - Learnable magnitude control
3. **Residual LoRA path** - Blends DoRA + LoRA dynamics

## Evolution Path
```
LoRA (2021) → DoRA (2024) → HyLoRADA (2026)
```

In [1]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['GRPC_VERBOSITY'] = 'ERROR'

In [2]:
# Clone repo
!git clone https://github.com/SadiaTabassum1216/hylorada.git
%cd hylorada

Cloning into 'hylorada'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 150 (delta 93), reused 104 (delta 47), pack-reused 0 (from 0)
Receiving objects: 100% (150/150), 174.82 KiB | 1.88 MiB/s, done.
Resolving deltas: 100% (93/93), done.
/kaggle/working/hylorada


In [3]:
!pip install -q transformers datasets accelerate tqdm optuna

In [4]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla P100-PCIE-16GB


---
## 2. Run Benchmark on Code

In [ ]:
!python run_benchmark.py --dataset code --methods lora dora hylorada --epochs 3 --output_dir ./results_code

PEFT Methods Benchmark
Model: Qwen/Qwen2-0.5B
Dataset: code
Methods: lora, dora, hylorada
Hyperparameters: epochs=3, batch=1, lr=0.0002
Max Length: 1024, LoRA Rank: 8

[1] Loading tokenizer...
tokenizer_config.json: 1.29kB [00:00, 6.10MB/s]
vocab.json: 2.78MB [00:00, 100MB/s]
merges.txt: 1.67MB [00:00, 143MB/s]
tokenizer.json: 7.03MB [00:00, 188MB/s]
[2] Loading data...
README.md: 33.2kB [00:00, 90.8MB/s]
README.md: 10.5kB [00:00, 42.4MB/s]
wikitext-2-raw-v1/test-00000-of-00001.pa(…): 100%|█| 733k/733k [00:01<00:00, 617
wikitext-2-raw-v1/train-00000-of-00001.p(…): 100%|█| 6.36M/6.36M [00:00<00:00, 7
wikitext-2-raw-v1/validation-00000-of-00(…): 100%|█| 657k/657k [00:00<00:00, 1.6
Generating train split: 100%|██| 36718/36718 [00:00<00:00, 748472.52 examples/s]
Generating validation split: 100%|█| 3760/3760 [00:00<00:00, 661323.56 examples/
    Dataset: WikiText-2 (fallback)
    Train: 1000, Test: 100

[3] Running LORA...
--------------------------------------------------
  Loading fresh 

---
## 3. Generate Qualitative Examples

In [ ]:
!python generate_examples.py \
    --checkpoint_dir ./results_code \
    --output comparison_examples.md

In [ ]:
# View the generated examples
with open('comparison_examples.md', 'r') as f:
    print(f.read())

---
## 5. View Results

In [ ]:
import json
import os

# Load WikiText results
# for results_dir in ['./results_wikitext', './results_code']:
for results_dir in ['./results_code']:
    results_file = os.path.join(results_dir, 'results.json')
    if os.path.exists(results_file):
        with open(results_file) as f:
            results = json.load(f)
        print(f"\n=== {results_dir} ===")
        for method, data in results.items():
            if 'perplexity' in data:
                print(f"{method}: PPL={data['perplexity']:.2f}, Params={data.get('trainable_params', 'N/A'):,}")

In [ ]:
# Download results for report
!zip -r hylorada_results.zip results_code comparison_examples.md